# Non-Negative MinTrace

Large collections of time series organized into structures at different aggregation levels often require their forecasts to follow their aggregation constraints and to be nonnegative, which poses the challenge of creating novel algorithms capable of coherent forecasts.

The `HierarchicalForecast` package provides a wide collection of Python implementations of hierarchical forecasting algorithms that follow nonnegative hierarchical reconciliation.

In this notebook, we will show how to use the `HierarchicalForecast` package to perform nonnegative reconciliation of forecasts on `Wiki2` dataset.

You can run these experiments using CPU or GPU with Google Colab.

<a href="https://colab.research.google.com/github/Nixtla/hierarchicalforecast/blob/main/nbs/examples/NonNegativeReconciliation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install hierarchicalforecast statsforecast datasetsforecast

## 1. Load Data

In this example we will use the `Wiki2` dataset. The following cell gets the time series for the different levels in the hierarchy, the summing dataframe  `S_df` which recovers the full dataset from the bottom level hierarchy and the indices of each hierarchy denoted by `tags`.

In [ ]:
import numpy as np
import pandas as pd

from datasetsforecast.hierarchical import HierarchicalData

In [ ]:
Y_df, S_df, tags = HierarchicalData.load('./data', 'Wiki2')
Y_df['ds'] = pd.to_datetime(Y_df['ds'])
S_df = S_df.reset_index(names="unique_id")

In [ ]:
Y_df.head()

,unique_id,ds,y
0,Total,2016-01-01,156508
1,Total,2016-01-02,129902
2,Total,2016-01-03,138203
3,Total,2016-01-04,115017
4,Total,2016-01-05,126042


In [ ]:
S_df.iloc[:5, :5]

,unique_id,de_AAC_AAG_001,de_AAC_AAG_010,de_AAC_AAG_014,de_AAC_AAG_045
0,Total,1,1,1,1
1,de,1,1,1,1
2,en,0,0,0,0
3,fr,0,0,0,0
4,ja,0,0,0,0


In [ ]:
tags

{'Views': array(['Total'], dtype=object),
 'Views/Country': array(['de', 'en', 'fr', 'ja', 'ru', 'zh'], dtype=object),
 'Views/Country/Access': array(['de_AAC', 'de_DES', 'de_MOB', 'en_AAC', 'en_DES', 'en_MOB',
        'fr_AAC', 'fr_DES', 'fr_MOB', 'ja_AAC', 'ja_DES', 'ja_MOB',
        'ru_AAC', 'ru_DES', 'ru_MOB', 'zh_AAC', 'zh_DES', 'zh_MOB'],
       dtype=object),
 'Views/Country/Access/Agent': array(['de_AAC_AAG', 'de_AAC_SPD', 'de_DES_AAG', 'de_MOB_AAG',
        'en_AAC_AAG', 'en_AAC_SPD', 'en_DES_AAG', 'en_MOB_AAG',
        'fr_AAC_AAG', 'fr_AAC_SPD', 'fr_DES_AAG', 'fr_MOB_AAG',
        'ja_AAC_AAG', 'ja_AAC_SPD', 'ja_DES_AAG', 'ja_MOB_AAG',
        'ru_AAC_AAG', 'ru_AAC_SPD', 'ru_DES_AAG', 'ru_MOB_AAG',
        'zh_AAC_AAG', 'zh_AAC_SPD', 'zh_DES_AAG', 'zh_MOB_AAG'],
       dtype=object),
 'Views/Country/Access/Agent/Topic': array(['de_AAC_AAG_001', 'de_AAC_AAG_010', 'de_AAC_AAG_014',
        'de_AAC_AAG_045', 'de_AAC_AAG_063', 'de_AAC_AAG_100',
        'de_AAC_AAG_110', 'de_AAC

We split the dataframe in train/test splits.

In [ ]:
Y_test_df = Y_df.groupby('unique_id', as_index=False).tail(7)
Y_train_df = Y_df.drop(Y_test_df.index)

## 2. Base Forecasts

The following cell computes the *base forecast* for each time series using the `AutoETS` model. Observe that `Y_hat_df` contains the forecasts but they are not coherent.

In [ ]:
%%capture
from statsforecast.models import AutoETS, Naive
from statsforecast.core import StatsForecast

In [ ]:
%%capture
fcst = StatsForecast(
    models=[AutoETS(season_length=7, model='ZAA'), Naive()], 
    freq='D', 
    n_jobs=-1
)
Y_hat_df = fcst.forecast(df=Y_train_df, h=7)

Observe that the `AutoETS` model computes negative forecasts for some series.

In [ ]:
Y_hat_df.query('AutoETS < 0')

,unique_id,ds,AutoETS,Naive
28,de_AAC_AAG_001,2016-12-25,-523.766907,340.0
29,de_AAC_AAG_001,2016-12-26,-245.337433,340.0
30,de_AAC_AAG_001,2016-12-27,-194.253815,340.0
33,de_AAC_AAG_001,2016-12-30,-315.425659,340.0
34,de_AAC_AAG_001,2016-12-31,-806.920105,340.0
...,...,...,...,...
1217,zh_AAC_AAG_033,2016-12-31,-86.466789,37.0
1345,zh_MOB,2016-12-26,-199.534882,1036.0
1346,zh_MOB,2016-12-27,-69.527260,1036.0
1352,zh_MOB_AAG,2016-12-26,-199.534882,1036.0


## 3. Non-Negative Reconciliation

The following cell makes the previous forecasts coherent and nonnegative using the `HierarchicalReconciliation` class.

In [ ]:
from hierarchicalforecast.methods import MinTrace
from hierarchicalforecast.core import HierarchicalReconciliation

In [ ]:
%%capture
reconcilers = [
    MinTrace(method='ols'),
    MinTrace(method='ols', nonnegative=True)
]
hrec = HierarchicalReconciliation(reconcilers=reconcilers)
Y_rec_df = hrec.reconcile(Y_hat_df=Y_hat_df, Y_df=Y_train_df,
                          S=S_df, tags=tags)

Observe that the nonnegative reconciliation method obtains nonnegative forecasts. 

In [ ]:
Y_rec_df

,unique_id,ds,AutoETS,Naive,AutoETS/MinTrace_method-ols,Naive/MinTrace_method-ols,AutoETS/MinTrace_method-ols_nonnegative-True,Naive/MinTrace_method-ols_nonnegative-True
0,Total,2016-12-25,94523.164062,95743.0,95852.000421,95743.0,9.664245e+04,95743.0
1,Total,2016-12-26,87734.367188,95743.0,89525.238276,95743.0,9.028857e+04,95743.0
2,Total,2016-12-27,87751.125000,95743.0,89638.119184,95743.0,9.056593e+04,95743.0
3,Total,2016-12-28,133237.968750,95743.0,131051.839057,95743.0,1.314028e+05,95743.0
4,Total,2016-12-29,126501.796875,95743.0,121214.048604,95743.0,1.218000e+05,95743.0
...,...,...,...,...,...,...,...,...
1388,zh_MOB_AAG_138,2016-12-27,62.049744,65.0,-147.399760,65.0,0.000000e+00,65.0
1389,zh_MOB_AAG_138,2016-12-28,54.934032,65.0,7.561682,65.0,4.397229e-15,65.0
1390,zh_MOB_AAG_138,2016-12-29,60.452618,65.0,114.253489,65.0,9.321380e+01,65.0
1391,zh_MOB_AAG_138,2016-12-30,50.356693,65.0,96.446754,65.0,7.565171e+01,65.0


In [ ]:
Y_rec_df.query('`AutoETS/MinTrace_method-ols_nonnegative-True` < 0')

,unique_id,ds,AutoETS,Naive,AutoETS/MinTrace_method-ols,Naive/MinTrace_method-ols,AutoETS/MinTrace_method-ols_nonnegative-True,Naive/MinTrace_method-ols_nonnegative-True


The free reconciliation method gets negative forecasts.

In [ ]:
Y_rec_df.query('`AutoETS/MinTrace_method-ols` < 0')

,unique_id,ds,AutoETS,Naive,AutoETS/MinTrace_method-ols,Naive/MinTrace_method-ols,AutoETS/MinTrace_method-ols_nonnegative-True,Naive/MinTrace_method-ols_nonnegative-True
56,de_DES,2016-12-25,-2553.932861,495.0,-3818.990043,495.0,0.000000e+00,495.0
57,de_DES,2016-12-26,-2155.228271,495.0,-3309.806933,495.0,1.909922e-30,495.0
58,de_DES,2016-12-27,-2720.993896,495.0,-3965.351121,495.0,1.140223e-13,495.0
60,de_DES,2016-12-29,-3429.432617,495.0,-3042.502484,495.0,3.049601e+02,495.0
61,de_DES,2016-12-30,-3963.202637,495.0,-3476.273292,495.0,2.877829e+02,495.0
...,...,...,...,...,...,...,...,...
1380,zh_MOB_AAG_036,2016-12-26,75.298317,115.0,-166.245228,115.0,0.000000e+00,115.0
1381,zh_MOB_AAG_036,2016-12-27,72.895554,115.0,-136.553950,115.0,1.699002e-14,115.0
1386,zh_MOB_AAG_138,2016-12-25,94.796623,65.0,-49.410174,65.0,0.000000e+00,65.0
1387,zh_MOB_AAG_138,2016-12-26,71.293983,65.0,-170.249562,65.0,0.000000e+00,65.0


## 4. Evaluation

The `HierarchicalForecast` package includes the `evaluate` function to evaluate the different hierarchies. We use `utilsforecast` to compute the mean absolute error.

In [ ]:
from hierarchicalforecast.evaluation import evaluate
from utilsforecast.losses import mse

In [ ]:
evaluation = evaluate(df = Y_rec_df.merge(Y_test_df, on=['unique_id', 'ds']),
                      tags = tags,
                      train_df = Y_train_df,
                      metrics = [mse],
                      benchmark="Naive")

evaluation.set_index(["level", "metric"]).filter(like='ETS')

,,AutoETS,AutoETS/MinTrace_method-ols,AutoETS/MinTrace_method-ols_nonnegative-True
level,metric,,,
Views,mse-scaled,0.735800,0.697371,0.675672
Views/Country,mse-scaled,1.190354,1.053631,0.994758
Views/Country/Access,mse-scaled,1.086102,1.133507,1.172270
Views/Country/Access/Agent,mse-scaled,1.067394,1.100215,1.127960
Views/Country/Access/Agent/Topic,mse-scaled,1.435105,1.381990,1.163428
Overall,mse-scaled,1.010801,0.977667,0.939286


Observe that the nonnegative reconciliation method performs better (lower error) than its unconstrained counterpart.

### References
- [Hyndman, R.J., & Athanasopoulos, G. (2021). "Forecasting: principles and practice, 3rd edition: 
Chapter 11: Forecasting hierarchical and grouped series.". OTexts: Melbourne, Australia. OTexts.com/fpp3 
Accessed on July 2022.](https://otexts.com/fpp3/hierarchical.html)
- [Wickramasuriya, S. L., Athanasopoulos, G., & Hyndman, R. J. (2019). \"Optimal forecast reconciliation for
    hierarchical and grouped time series through trace minimization\". Journal of the American Statistical Association, 
    114 , 804–819. doi:10.1080/01621459.2018.1448825.](https://robjhyndman.com/publications/mint/).
- [Wickramasuriya, S.L., Turlach, B.A. & Hyndman, R.J. (2020). \"Optimal non-negative
    forecast reconciliation". Stat Comput 30, 1167–1182, 
    https://doi.org/10.1007/s11222-020-09930-0](https://robjhyndman.com/publications/nnmint/).